## Imports

In [58]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm


from sklearn.metrics import (
    accuracy_score,
    # ENHANCEMENT 1
    precision_score,
    recall_score,
    f1_score
)

## Dataset

In [59]:
data_df = pd.read_csv('./data/winequality-red.csv')

In [60]:
data_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [61]:
# how many features?
len(data_df.columns) - 1

11

In [62]:
# how many labels? If yours is a binary classification task, then you'll have 2 labels.
data_df.quality.unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [63]:
# convert these quaity measures to labels (0 to 5)
def get_label(quality):
    if quality == 3:
        return 0
    elif quality == 4:
        return 1
    elif quality == 5:
        return 2
    elif quality == 6:
        return 3
    elif quality == 7:
        return 4
    else:
        return 5

labels = data_df['quality'].apply(get_label)

# normalize data
data_df = (data_df - data_df.mean()) / data_df.std()
data_df['label'] = labels

In [64]:
data_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label
0,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,-0.959946,-0.787576,2
1,-0.298454,1.966827,-1.391037,0.043403,0.223805,0.872365,0.624168,0.028252,-0.719708,0.128910,-0.584594,-0.787576,2
2,-0.298454,1.296660,-1.185699,-0.169374,0.096323,-0.083643,0.228975,0.134222,-0.331073,-0.048074,-0.584594,-0.787576,2
3,1.654339,-1.384011,1.483689,-0.453077,-0.264878,0.107558,0.411372,0.664069,-0.978798,-0.461036,-0.584594,0.450707,3
4,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,-0.959946,-0.787576,2


In [65]:
# sumamry statistics of the data
data_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label
count,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1599.000000
mean,-1.570643e-14,-1.590973e-15,6.045057e-15,-1.344696e-15,4.924731e-15,-7.724347e-17,1.619856e-16,4.946064e-13,-5.937923e-15,-2.175036e-15,2.580411e-14,1.081756e-15,2.636023
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.807569
min,-2.136377e+00,-2.277567e+00,-1.391037e+00,-1.162333e+00,-1.603443e+00,-1.422055e+00,-1.230199e+00,-3.537625e+00,-3.699244e+00,-1.935902e+00,-1.898325e+00,-3.264143e+00,0.000000
25%,-7.004996e-01,-7.696903e-01,-9.290275e-01,-4.530767e-01,-3.711129e-01,-8.484502e-01,-7.438076e-01,-6.075656e-01,-6.549356e-01,-6.380200e-01,-8.661079e-01,-7.875763e-01,2.000000
50%,-2.410190e-01,-4.367545e-02,-5.634264e-02,-2.402999e-01,-1.798892e-01,-1.792441e-01,-2.574163e-01,1.759533e-03,-7.210449e-03,-2.250577e-01,-2.092427e-01,4.507074e-01,3.000000
75%,5.056370e-01,6.264921e-01,7.650078e-01,4.340257e-02,5.382858e-02,4.899619e-01,4.721707e-01,5.766445e-01,5.757422e-01,4.238832e-01,6.352984e-01,4.507074e-01,3.000000
max,4.353787e+00,5.876138e+00,3.742403e+00,9.192806e+00,1.112355e+01,5.365606e+00,7.372847e+00,3.678904e+00,4.526866e+00,7.916200e+00,4.201138e+00,2.927275e+00,5.000000


## Load this dataset for training a neural network

In [66]:
# The dataset class
class WineDataset(Dataset):

    def __init__(self, data_df):
        self.data_df = data_df
        self.features = []
        self.labels = []
        for _, i in data_df.iterrows():
            self.features.append([i['fixed acidity'], i['volatile acidity'], i['citric acid'], i['residual sugar'], i['chlorides'], i['free sulfur dioxide'], i['total sulfur dioxide'], i['density'], i['pH'], i['sulphates'], i['alcohol']])
            self.labels.append(i['label'])

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = self.features[idx]
        features = torch.FloatTensor(features)

        labels = torch.tensor(int(self.labels[idx]), dtype = torch.long)

        return {'labels': labels, 'features': features}

wine_dataset = WineDataset(data_df)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(wine_dataset, [0.8, 0.1, 0.1])

# The dataloader
train_dataloader = DataLoader(train_dataset, batch_size = 4, shuffle = True, num_workers = 0)
val_dataloader = DataLoader(val_dataset, batch_size = 4, shuffle = False, num_workers = 0)
test_dataloader = DataLoader(test_dataset, batch_size = 4, shuffle = False, num_workers = 0)

In [67]:
# peak into the dataset
for i in wine_dataset:
    print(i)
    break

{'labels': tensor(2), 'features': tensor([-0.5282,  0.9616, -1.3910, -0.4531, -0.2436, -0.4660, -0.3790,  0.5581,
         1.2882, -0.5790, -0.9599])}


## Neural Network

In [68]:
# change the device to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [69]:
class WineModel(torch.nn.Module):

    def __init__(self):
        super(WineModel, self).__init__()
                                            # ENHANCEMENT 4 (Larger right number)
        self.linear1 = torch.nn.Linear(11, 200)
        self.activation = torch.nn.ReLU()
                                      # ENHANCEMENT 3
        self.linear2 = torch.nn.Linear(200, 400) # ENHANCEMENT 4 (Larger numbers)
        self.linear3 = torch.nn.Linear(400, 6) # ENHANCEMENT 4 (Larger left number)

        self.softmax = torch.nn.Softmax(dim =1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        
        # ENHANCEMENT 3
        x = self.activation(x)
        x = self.linear3(x)
        x = self.softmax(x)
        return x

winemodel = WineModel().to(device)

## Training

In [70]:
# Define and the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
                                            # ENHANCEMENT 2
optimizer = AdamW(winemodel.parameters(), lr = 1e-3)

In [71]:
# Lets define the training steps

def accuracy(preds, labels):
    preds = torch.argmax(preds, dim=1).flatten()
    labels = labels.flatten()
    return torch.sum(preds == labels) / len(labels)

def train(model, data_loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for d in tqdm(data_loader):
        inputs = d['features'].to(device)
        labels = d['labels'].to(device)
        outputs = winemodel(inputs)

        _, preds = torch.max(outputs, dim=1)
        loss = criterion(outputs, labels)
        acc = accuracy(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(data_loader), epoch_acc / len(data_loader)

# Lets define the testing steps
def evaluate(model, data_loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    # ENHANCEMENT 1
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()
    with torch.no_grad():
        for d in data_loader:
            inputs = d['features'].to(device)
            labels = d['labels'].to(device)
            outputs = winemodel(inputs)

            _, preds = torch.max(outputs, dim=1)
            loss = criterion(outputs, labels)
            acc = accuracy(outputs, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            # ENHANCEMENT 1
            precision = precision_score(labels.cpu(), preds.cpu(), average='weighted', zero_division=0)
            recall = recall_score(labels.cpu(), preds.cpu(), average='weighted',zero_division=0)
            f1 = f1_score(labels.cpu(), preds.cpu(), average='weighted',zero_division=0)
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1
            
    num_batches = len(data_loader)
    return (
        epoch_loss / num_batches,
        epoch_acc / num_batches,
        
        # ENHANCEMENT 1
        epoch_precision / num_batches, 
        epoch_recall / num_batches,
        epoch_f1 / num_batches
    )



In [72]:
# Let's train our model
                   # ENHANCEMENT 2
for epoch in range(500):
    train_loss, train_acc = train(winemodel, train_dataloader, optimizer, criterion)
    # ENHANCEMENT 1
    valid_loss, valid_acc, valid_precision, valid_recall, valid_f1  = evaluate(winemodel, val_dataloader, criterion)

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% | Val. Precision: {valid_precision *100:.3f} | Val. Recall: {valid_recall*100:.3f} | Val. F1-score: {valid_f1*100:.3f} |')

  0%|          | 0/320 [00:00<?, ?it/s]

| Epoch: 01 | Train Loss: 1.491 | Train Acc: 54.84% | Val. Loss: 1.452 | Val. Acc: 59.38% | Val. Precision: 55.937 | Val. Recall: 59.375 | Val. F1-score: 55.143 |


  0%|          | 0/320 [00:00<?, ?it/s]

| Epoch: 02 | Train Loss: 1.465 | Train Acc: 57.66% | Val. Loss: 1.454 | Val. Acc: 57.50% | Val. Precision: 53.177 | Val. Recall: 57.500 | Val. F1-score: 52.899 |


  0%|          | 0/320 [00:00<?, ?it/s]

...

| Epoch: 498 | Train Loss: 1.221 | Train Acc: 82.27% | Val. Loss: 1.394 | Val. Acc: 65.00% | Val. Precision: 67.240 | Val. Recall: 65.000 | Val. F1-score: 61.899 |


  0%|          | 0/320 [00:00<?, ?it/s]

| Epoch: 499 | Train Loss: 1.231 | Train Acc: 81.25% | Val. Loss: 1.395 | Val. Acc: 64.38% | Val. Precision: 64.948 | Val. Recall: 64.375 | Val. F1-score: 61.396 |


  0%|          | 0/320 [00:00<?, ?it/s]

| Epoch: 500 | Train Loss: 1.234 | Train Acc: 81.02% | Val. Loss: 1.367 | Val. Acc: 67.50% | Val. Precision: 65.990 | Val. Recall: 67.500 | Val. F1-score: 63.670 |


# Lab Enhancements
* These tasks are additional enhancements with less guidance.
* Report results means give us the accuracy, precision, recall and F1-score.


## Enhancement 1: The current code does not actually evaluate the model on the test set, but it only evaluates it on the val set. When you write papers, you would ideally split the dataset into train, val and test. Train and val are both used in training, and the model trained on the training data, and  evaluated on the val data. So why do we need test split? We report our results on the test split in papers. Also, we do cross-validation on the train/val split (covered in later labs).

## Report the results of the model on the test split. (Hint: It would be exactly like the evaluation on the val dataset, except it would be done on the test dataset.)

In [73]:
test_loss, test_acc, test_prec, test_rec, test_f1 = evaluate(winemodel, test_dataloader, criterion)

print(f'| Test. Loss: {test_loss:.3f} | Test. Acc: {test_acc*100:.2f}% | Test. Precision: {test_prec:.3f} | Test. Recall: {test_rec:.3f} | Test. F1-score: {test_f1:.3f} |')

| Test. Loss: 1.461 | Test. Acc: 56.67% | Test. Precision: 0.586 | Test. Recall: 0.567 | Test. F1-score: 0.539 |


## Enhancement 2: Increase the number of epochs (and maybe the learning rate). Does the accuracy on the test set increase? Is there a significant difference between the test accuracy and the train accuracy? If yes, why?

## Enhancement 3: Increase the depth of your model (add more layers). Report the parts of the model definition you had to update. Report results.

## Enhancement 4: Increase the width of your model's layers. Report the parts of the model definition you had to update. Report results.

## Enhancement 5: Choose a new dataset from the list below. Search the Internet and download your chosen dataset (many of them could be available on kaggle). Adapt your model to your dataset. Train your model and record your results.

   * cancer_dataset          - Breast cancer dataset.
   * crab_dataset            - Crab gender dataset.
   * glass_dataset           - Glass chemical dataset.
   * iris_dataset            - Iris flower dataset.
   * ovarian_dataset         - Ovarian cancer dataset.
   * thyroid_dataset         - Thyroid function dataset.